<div class = "alert alert-info">

# Allégement du fichier des émissions
    

Il y a quatre champs :
```
- id             : identifiant du modèle de véhicule, c'est le même que celui de la table des véhicules
- score_smog1    : score qualité (1 à 10) en terme de particules pour énergie 1 (-1 si pas de score)
- score_smog2    : score qualité (1 à 10) en terme de particules pour énergie 2 (-1 si pas de score)
- score_smartway : label de qualité environnementale "SmartWay" ('NON', 'SMARTWAY' ou 'SMARTWAY ELITE')
```
    

In [6]:
import csv

def enlever_doublons_liste(avec_doublons):
    sans_doublons = []
    for element in avec_doublons:
        if element not in sans_doublons:
            sans_doublons.append(element)
    return sans_doublons

def enlever_doublons_ids(avec_doublons):
    sans_doublons = []
    sans_doublons_ids = []
    for element in avec_doublons:
        if element['id'] not in sans_doublons_ids:
            sans_doublons.append(element)
            sans_doublons_ids.append(element['id'])
    return sans_doublons    

def importer_csv(chemin_fichier):
    liste_dicos = []
    fichier_csv = open(chemin_fichier, 'r', encoding = 'UTF-8', newline = '')
    lecteur_csv = csv.DictReader(fichier_csv, delimiter = ',')
    for element in lecteur_csv :
        liste_dicos.append(dict(element))
    fichier_csv.close()
    return liste_dicos

def enregistrer_en_csv(chemin_fichier, table):
    with open(chemin_fichier, 'w', newline='', encoding = 'UTF-8') as csvfile:
        fieldnames = table[0].keys()
        writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
        writer.writeheader()
        for row in table:
            writer.writerow(row)

def alleger_fichier_emissions():
    emissions = importer_csv('.\\emissions\\emissions.csv')
    emissions_light = []
    cles_retenues = ['id', 'score', 'scoreAlt', 'smartwayScore']
    cles_modifiees = {'id':'id', 'score':'score_smog1', 'scoreAlt':'score_smog2', 
                      'smartwayScore':'score_smartway'}
    
    for em in emissions :
        em_light = {cles_modifiees[cle]:val for cle, val in em.items() if cle in cles_retenues}
        emissions_light.append(em_light)
    
    for em in emissions_light :
        for cle, val in em.items():
            if cle == 'score_smartway':
                if val == '-1':
                    em[cle] = 'NON'
                elif val == '1':
                    em[cle] = 'SMARTWAY'
                elif val == '2':
                    em[cle] = 'SMARTWAY ELITE'
    
    emissions_light = enlever_doublons_liste(emissions_light)
    emissions_light = enlever_doublons_ids(emissions_light)

    enregistrer_en_csv('.\\emissions\\emissions_light.csv', emissions_light)

In [ ]:
alleger_fichier_emissions()


<div class = "alert alert-info">

# Allégement du fichier de véhicules

<div class = "alert alert-info" >

- PHEV : Plugin Hybrid Electric Vehicle
- CNG : Compressed Natural Gas
- FFV : Flexi Fuel Vehicle    
- EPA = Environnmental Protection Agency
- MPG : Miles Per Gallon
- fuelType : reprend fuelType1 et fuelType2 donc inutile
- atvType : même problématique
- comb08 : toujours différent de `'0'`
- combA08 : `'0'` lorsque fuelType2 est `''`




<pre>
- Modèle :
    - id           : identifiant
    - make         : fabricant
    - model        : nom du modèle
    - year         : année de production du modèle
   
- Spécifications techniques :
    - cylinders    : nombre de cylindres
    - displ        : cylindrée (L)
    - trany        : transmission
    - VClass       : classe du véhicule selon l'EPA (Environnment Protection Agency)
    - startStop    : technologie Start-Stop présente ('Y'/'N'/'')
    - fuelType1    : énergie principale
    - fuelType2    : énergie secondaire
    - fuelType 
    
- Consommations et émissions :
    - atvType      : type d'énergie alternative
    - city08       : consommation (L/100km) en cycle urbain pour l'énergie principale (*)
    - cityA08      : consommation (L/100km) en cycle urbain pour l'énergie alternative (*) 
    - cityE        : consommation (kW.h/100km) en cycle urbain pour l'énergie électrique
    - highway08    : consommation (L/100km) sur autoroute pour l'énergie principale (*)
    - highwayA08   : consommation (L/100km) sur autoroute pour l'énergie alternative (*)
    - highwayE     : consommation (kW.h/100km) sur autoroute pour l'énergie électrique
    - comb08       : consommation (L/100km) en cycle mixte pour l'énergie principale (*)
    - combA08      : consommation (L/100km) en cycle mixte pour l'énergie alternative (*) 
    - combE        : consommation (kW.h/100km) en cycle mixte pour l'énergie électrique
    - co2          : émissions de CO2 (g/km) pour l'énergie principale
    - co2A         : émissions de CO2 (g/km) pour l'énergie alternative    
    
(*) : si besoin converties en équivalent L/100km 
</pre>

In [18]:
import csv

def importer_csv(chemin_fichier):
    liste_dicos = []
    fichier_csv = open(chemin_fichier, 'r', encoding = 'UTF-8', newline = '')
    lecteur_csv = csv.DictReader(fichier_csv, delimiter = ',')
    for element in lecteur_csv :
        liste_dicos.append(dict(element))
    fichier_csv.close()
    return liste_dicos

def enregistrer_en_csv(chemin_fichier, table):
    with open(chemin_fichier, 'w', newline='', encoding = 'UTF-8') as csvfile:
        fieldnames = table[0].keys()
        writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
        writer.writeheader()
        for row in table:
            writer.writerow(row)

def ccf(x):
    return (100*3.78541)/(1.60934*float(x))

def alleger_fichier():
    cars = importer_csv('./datas_voitures/vehicles/vehicles.csv')
    cars_light = []
    cles_retenues = ['id', 'make', 'model', 'year', 'atvType', 
                     'cylinders', 'displ', 'trany', 'VClass', 'startStop', 'fuelType1', 'fuelType2', 'fuelType', 
                     'city08', 'cityA08', 'cityE', 'highway08', 'highwayA08', 'highwayE',
                     'comb08', 'combA08', 'combE', 'co2', 'co2A']
    for car in cars :
        car_light = {cle:val for cle, val in car.items() if cle in cles_retenues}
        cars_light.append(car_light)
    for car in cars_light :
        for key in ['city08', 'cityA08', 'highway08', 'highwayA08', 'comb08', 'combA08'] :
            if car[key] != '0':
                car[key] = str(round((100*3.78541)/(1.60934*float(car[key])), 1))
        for key in ['co2', 'co2A'] :
            if car[key] != '-1':
                car[key] = str(round(float(car[key])/1.60934, 1))
    enregistrer_en_csv('.\\vehicles\\vehicles_light.csv', cars_light)
    
def super_alleger_fichier():
    cars = importer_csv('./datas_voitures/vehicles/vehicles.csv')
    cars_light = []
    cles_retenues = ['id', 'make', 'model', 'year', 
                     'cylinders', 'displ', 'trany', 'VClass', 'startStop', 'fuelType1', 'fuelType2', 
                     'comb08', 'combA08', 'combE', 'co2', 'co2A']
    cles_modifiees = {'id':'id',
                      'make' : 'fabricant', 
                      'model' : 'nom_modele', 
                      'year' : 'annee', 
                      'cylinders' : 'nb_cylindres', 
                      'displ' : 'cylindree', 
                      'trany' : 'transmission', 
                      'VClass' : 'categorie', 
                      'startStop' : 'ss', 
                      'fuelType1' : 'energie1', 
                      'fuelType2' : 'energie2', 
                      'comb08' : 'consoMixte1', 
                      'combA08' : 'consoMixte2', 
                      'combE' : 'consoMixteE', 
                      'co2' : 'CO2', 
                      'co2A' : 'CO2A'}
    for car in cars :
        car_light = {cles_modifiees[cle]:val for cle, val in car.items() if cle in cles_retenues}
        cars_light.append(car_light)
    for car in cars_light :
        for key in ['consoMixte1', 'consoMixte2'] :
            if car[key] != '0':
                car[key] = str(round((100*3.78541)/(1.60934*float(car[key])), 1))
        for key in ['CO2', 'CO2A'] :
            if car[key] != '-1':
                car[key] = str(round(float(car[key])/1.60934, 1))
        if car['cylindree'] == '' or car['cylindree'] == 'NA':
            car['cylindree'] = '0'
                
    cars_light = enlever_doublons_liste(cars_light)
    cars_light = enlever_doublons_ids(cars_light)                
                
                
    enregistrer_en_csv('./datas_voitures/vehicles/vehicles_super_light.csv', cars_light)
    


In [19]:
super_alleger_fichier()

In [ ]:
cars_light = importer_csv('.\\vehicles\\vehicles_light.csv')

In [ ]:
#if car['combA08'] != '0'
L = [(car['fuelType1'], car['fuelType2'], car['atvType']) for car in cars_light]
S = set(L)
S

In [ ]:
#if car['combA08'] != '0'
L = [(car['fuelType1'], car['fuelType2'], car['fuelType']) for car in cars_light]
S = set(L)
S

In [ ]:
import random
cars_super_light = importer_csv('.\\vehicles\\vehicles_super_light.csv')
cars_super_light_shorten = random.sample(cars_super_light, 500)
print(list(cars_super_light_shorten[0].keys()))

In [ ]:
#if car['combA08'] != '0'
cars_super_light_shorten = random.sample(cars_super_light, 500)
L = [(car['consoMixte1'], car['energie1']) for car in cars_super_light]
S = set(L)
S

In [ ]:
#if car['combA08'] != '0'
cars_super_light_shorten = random.sample(cars_super_light, 500)
L = [(car['consoMixte2'], car['energie2']) for car in cars_super_light]
S = set(L)
S